In [ ]:
'''Plan:
1. Load libraries, load data.
2. Preliminary EDA, dealing with missing values.
3. EDA, deleting variables.
4. Sample formation.
5. Feature scaling.
6. Model fitting.
7. Performance evaluation.
8. Predictions.
'''

In [5]:
import numpy as np
import pandas as pd
import os, warnings, random, time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBClassifier


In [7]:
#os.getcwd()

train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
print(train.shape)
train.head(3)

(957919, 120)


,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,...,-12.2280,1.7482,1.90960,-7.1157,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.1420,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,...,-5.7688,1.2042,0.26290,8.1312,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1


In [8]:
train.describe()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
count,957919.000000,942672.000000,942729.000000,942428.000000,942359.000000,942514.000000,942398.000000,942415.000000,9.425460e+05,9.426700e+05,...,942554.000000,942420.000000,942509.000000,942686.000000,942481.000000,942360.000000,9.423300e+05,942512.000000,942707.000000,957919.000000
mean,478959.000000,0.090201,0.345964,4068.744207,0.201214,0.304869,-0.071458,1620.843815,3.771642e+05,1.806054e+15,...,-19.926398,2.074530,23.885245,1.748777,63152.973540,1.208876,4.276905e+16,3959.204669,0.559267,0.498492
std,276527.540593,0.043564,0.146251,6415.829440,0.212510,0.145343,2.123777,1276.281403,3.454325e+05,2.335204e+15,...,18.578439,0.895793,45.581360,10.088848,92435.016241,0.114959,6.732441e+16,3155.991777,0.408426,0.499998
min,0.000000,-0.149910,-0.019044,-9421.700000,-0.082122,-0.006990,-12.791000,-224.800000,-2.984300e+04,-1.153300e+15,...,-105.860000,0.277040,-27.691000,-26.589000,-81977.000000,0.905270,-8.944400e+15,-415.240000,-0.151240,0.000000
25%,239479.500000,0.070227,0.283050,418.430000,0.035086,0.240520,-1.120700,481.545000,9.120900e+04,1.153100e+13,...,-28.812000,1.487700,-0.628880,-4.473975,2443.200000,1.146800,2.321100e+14,1306.200000,0.276560,0.000000
50%,478959.000000,0.090135,0.389100,1279.500000,0.137000,0.327790,-0.380110,1446.100000,2.896700e+05,5.043050e+14,...,-14.636000,1.662100,1.727700,0.885710,19479.000000,1.177200,1.327500e+16,3228.000000,0.473440,0.000000
75%,718438.500000,0.116500,0.458450,4444.400000,0.297100,0.412830,0.921940,2495.900000,5.605600e+05,3.103100e+15,...,-5.325300,2.522325,18.991000,6.840775,88488.000000,1.242000,5.278700e+16,6137.900000,0.746210,1.000000
max,957918.000000,0.415170,0.518990,39544.000000,1.319900,0.554750,11.202000,5426.600000,1.913700e+06,1.042400e+16,...,1.613400,4.565900,217.840000,47.757000,526050.000000,1.886700,3.249900e+17,13151.000000,2.743600,1.000000


In [35]:
#df = train.sample(n=100000)
df = train.copy()
df.shape

(957919, 120)

In [36]:
un_colval = pd.DataFrame([[x,len(df[x].unique())] for x in df.columns], columns = ['colname', 'n_unique'])
un_colval.loc[un_colval.n_unique < 100]
# all columns contain numerical features

,colname,n_unique
119,claim,2


In [37]:
#df.info(verbose=True, null_counts=True)
df = df.dropna()
df.shape

(359464, 120)

In [38]:
### 

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
1,1,0.100900,0.299610,11822.000,0.276500,0.459700,-0.837330,1721.900,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.000,0.28359,0
8,8,-0.008948,0.338020,503.810,0.601520,0.261760,0.735380,1614.000,408030.0,7.684400e+14,...,-9.5142,2.2176,-1.23750,7.07130,1030.80,1.2040,5.290300e+15,1968.100,0.14593,0
9,9,0.126230,0.173960,1662.000,0.038081,0.000485,0.059909,296.070,-8035.3,4.526300e+15,...,-1.9684,3.9595,19.16900,-0.24498,838.39,1.1968,2.567000e+17,4914.900,0.51976,0
11,11,0.078650,0.387450,1025.900,0.082626,0.267070,-1.476200,2170.100,513560.0,9.716600e+12,...,-17.1010,1.5954,121.02000,-9.33300,45667.00,1.2490,5.059300e+15,5422.300,0.57097,0
14,14,0.072488,0.473410,-22.274,0.002846,0.384460,3.548400,-40.267,739910.0,4.615400e+15,...,-10.3290,1.6152,195.36000,1.05310,1806.40,1.2229,6.537300e+15,591.480,0.30038,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957910,957910,0.102000,0.426460,-187.990,0.109040,-0.004372,4.111300,377.860,7770.8,4.943600e+13,...,-5.3609,3.6449,-0.63353,-15.28400,213470.00,1.1392,7.059700e+16,3190.300,0.59345,0
957911,957911,0.072651,-0.011568,1916.800,0.038723,0.330100,-0.153940,681.880,104710.0,6.897600e+14,...,-50.5350,1.5725,18.20900,2.53950,269000.00,1.5845,2.797400e+14,29.983,0.46625,0
957912,957912,0.052017,0.305900,3319.900,0.026973,-0.003338,6.328600,2235.300,652460.0,-9.234500e+12,...,-13.0460,1.2816,2.64550,0.55064,9424.40,1.2938,-1.541700e+15,814.420,0.24652,0
957914,957914,0.098979,0.202530,2192.100,0.348010,0.276430,-0.840650,2214.200,26246.0,1.061600e+15,...,-51.2970,1.7323,20.76300,14.74400,27415.00,1.2160,2.139400e+13,2612.000,0.42597,0
